In [1]:
# Sagar
#Install necessary libraries
import pickle
from tensorflow.keras.models import load_model
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from moviepy.editor import *
%matplotlib inline
from sklearn.model_selection import train_test_split

In [ ]:
# load previously saved model. In this case LCRN50
LCRN50 = load_model('LCRN50.h5')
# summarize model.
LCRN50.summary()

In [3]:
#Lets create a function that takes URL from us and downloads the video in a folder. 
#we will us "pafy" library. it will download video with other essential informations.
def Video_To_Download(youtube_video_url, output_directory): 
     # Create a video object 
     video = pafy.new(youtube_video_url)
     # Getting title of video
     title = video.title 
     # We will take best quality available
     video_best = video.getbest() 
     # This is our output path for saving videos
     output_file_path = f'{output_directory}/{title}.mp4'
 
     # Downloading and storing
     video_best.download(filepath = output_file_path, quiet = True)
     return title

In [4]:
# We create a folder "Downloaded_Videos" to save out videos
test_videos_directory = 'Downloaded_Videos'
os.makedirs(test_videos_directory, exist_ok = True)

# Long downloadinng process aye. It takes ages. its just to test if we can download or not
# Tis is Kobe Bryant doing his top 10 highschool shots.
#video_title = Video_To_Download('https://www.youtube.com/watch?v=LFtTJdfwahQ', test_videos_directory)

# Path of downloaded video
#Video_Path = f'{test_videos_directory}/{video_title}.mp4'


# Problem with youtube videos is that the action can be a bit down the video. so taking
# just 10 frames maynot be enough.
# we will take 10frames of the video.
SEQUENCE_LENGTH =10

# For the sake of computational power and "ime is limited" we will only work with 64*64 images
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# these are the classes we are working with for this project
CLASSES_LIST = ['BaseballPitch',
  'Basketball',
  'HorseRace',
  'HorseRiding',
  'HulaHoop',
  'JavelinThrow',
  'JugglingBalls',
  'PullUps',
  'Punch',
  'PushUps',]

In [5]:
def Make_Prediction(video_file_path, SEQUENCE_LENGTH):
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting width and height of our video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a list to store those frames
    frames_list = []
    predicted_class_name = ''

    # Count number of frames from video
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # find interval so that frames can be added to list
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

    # go through as per our defined sequence number
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        # read each frame
        success, frame = video_reader.read() 
        # break if unreadable
        if not success:
            break
        # change dimension as per our interest.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))        
        # Normalize 
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)

    # Pass frames to our trained network
    predicted_labels_probabilities = LCRN50.predict(np.expand_dims(frames_list, axis = 0))[0]

    # Get index of class which has highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]
    
    # Display the predicted action and prediction confidence in %.
    print(f'Action Predicted: {predicted_class_name}\nConfidence Level: {predicted_labels_probabilities[predicted_label]}')
    video_reader.release()

In [ ]:
# We load our url here.
# find a video from youtube
Video_Action = Video_To_Download('https://www.youtube.com/link', test_videos_directory)

# Video path
Video_Path = f'{test_videos_directory}/{Video_Action}.mp4'

# Predict 1 action class for entire video.
Make_Prediction(Video_Path, SEQUENCE_LENGTH)
